# OpenApiPy Jupyter Sample

In this Jupyter notebook we will use the Python package "ctrader-open-api" to get daily trend bars data from cTrader Open API.

Let's start.

If you haven't already installed the "ctrader-open-api" package, run the next code cell to install it via pip:

In [ ]:
!pip install ctrader-open-api

Then we have to import all necessary types:

In [ ]:
from ctrader_open_api import Client, Protobuf, TcpProtocol, Auth, EndPoints
from ctrader_open_api.messages.OpenApiCommonMessages_pb2 import *
from ctrader_open_api.messages.OpenApiCommonMessages_pb2 import *
from ctrader_open_api.messages.OpenApiMessages_pb2 import *
from ctrader_open_api.messages.OpenApiModelMessages_pb2 import *
from twisted.internet import reactor
import json
import datetime

Now we use the "credentials-dev.json" file to get your Open API application credentials.
Be sure to populate it with your API application credentials and access token before running next cell:

In [ ]:
credentialsFile = open("credentials-dev.json")
credentials = json.load(credentialsFile)

Then we will create a client based our selected host type:

In [ ]:
host = EndPoints.PROTOBUF_LIVE_HOST if credentials["HostType"].lower() == "live" else EndPoints.PROTOBUF_DEMO_HOST
client = Client(host, EndPoints.PROTOBUF_PORT, TcpProtocol)

Now let's set the symbol name that you want to use, it must match to one of your trading account symbol names:

In [ ]:
symbolName = "EURUSD"

We will use the below list to store the daily bars data:

In [ ]:
dailyBars = []

We will use below method to transform the Open API trend bar to a tuple with bar open time, open price, high price, low price, and close price:

In [ ]:
def transformTrendbar(trendbar):
    openTime = datetime.datetime.fromtimestamp(trendbar.utcTimestampInMinutes * 60)
    openPrice = (trendbar.low + trendbar.deltaOpen) / 100000.0
    highPrice = (trendbar.low + trendbar.deltaHigh) / 100000.0
    lowPrice = trendbar.low / 100000.0
    closePrice = (trendbar.low + trendbar.deltaClose) / 100000.0
    return (openTime, openPrice, highPrice, lowPrice, closePrice, trendbar.volume)

OpenApiPy uses Twisted to work asynchronously, so the execution flow is not sequential, we have to use a series of callbacks.

The first callback we will use is the client "Connected" callback, its got triggered when client is connected and we should use it to send the Application authentication request.

We will then use the Application authentication request returned Twisted deferred to set response callbacks, when we received the "ProtoOAApplicationAuthRes" the deferred assigned callback chain will be triggered.

After we authenticated our API application then we will send an account authentication response, for the account ID you set on credentials file.

Be sure your account type and host type match, otherwise account authentication will fail.

Then we continue by using the retuned deferred to get trend bars data.

Now lets set the client callbacks:

In [ ]:
def trendbarsResponseCallback(result):
    print("\nTrendbars received")
    trendbars = Protobuf.extract(result)
    barsData = list(map(transformTrendbar, trendbars.trendbar))
    global dailyBars
    dailyBars.clear()
    dailyBars.extend(barsData)
    print("\ndailyBars length:", len(dailyBars))
    
def symbolsResponseCallback(result):
    print("\nSymbols received")
    symbols = Protobuf.extract(result)
    global symbolName
    symbolsFilterResult = list(filter(lambda symbol: symbol.symbolName == symbolName, symbols.symbol))
    if len(symbolsFilterResult) == 0:
        raise Exception(f"There is symbol that matches to your defined symbol name: {symbolName}")
    elif len(symbolsFilterResult) > 1:
        raise Exception(f"More than one symbol matched with your defined symbol name: {symbolName}, match result: {symbolsFilterResult}")
    symbol = symbolsFilterResult[0]
    request = ProtoOAGetTrendbarsReq()
    request.symbolId = symbol.symbolId
    request.ctidTraderAccountId = credentials["AccountId"]
    request.period = ProtoOATrendbarPeriod.D1
    # We set the from/to time stamps to 50 weeks, you can load more data by sending multiple requests
    # Please check the ProtoOAGetTrendbarsReq documentation for more detail
    request.fromTimestamp = int((datetime.datetime.utcnow() - datetime.timedelta(weeks=50)).timestamp()) * 1000
    request.toTimestamp = int(datetime.datetime.utcnow().timestamp()) * 1000
    deferred = client.send(request)
    deferred.addCallbacks(trendbarsResponseCallback, onError)
    
def accountAuthResponseCallback(result):
    print("\nAccount authenticated")
    request = ProtoOASymbolsListReq()
    request.ctidTraderAccountId = credentials["AccountId"]
    request.includeArchivedSymbols = False
    deferred = client.send(request)
    deferred.addCallbacks(symbolsResponseCallback, onError)
    
def applicationAuthResponseCallback(result):
    print("\nApplication authenticated")
    request = ProtoOAAccountAuthReq()
    request.ctidTraderAccountId = credentials["AccountId"]
    request.accessToken = credentials["AccessToken"]
    deferred = client.send(request)
    deferred.addCallbacks(accountAuthResponseCallback, onError)

def onError(client, failure): # Call back for errors
    print("\nMessage Error: ", failure)

def disconnected(client, reason): # Callback for client disconnection
    print("\nDisconnected: ", reason)

def onMessageReceived(client, message): # Callback for receiving all messages
    if message.payloadType in [ProtoHeartbeatEvent().payloadType, ProtoOAAccountAuthRes().payloadType, ProtoOAApplicationAuthRes().payloadType, ProtoOASymbolsListRes().payloadType, ProtoOAGetTrendbarsRes().payloadType]:
        return
    print("\nMessage received: \n", Protobuf.extract(message))
    
def connected(client): # Callback for client connection
    print("\nConnected")
    request = ProtoOAApplicationAuthReq()
    request.clientId = credentials["ClientId"]
    request.clientSecret = credentials["Secret"]
    deferred = client.send(request)
    deferred.addCallbacks(applicationAuthResponseCallback, onError)
    
# Setting optional client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

The last step is to run our client service, it will run inside Twisted reactor loop asynchronously:

In [ ]:
# Starting the client service
client.startService()
# Run Twisted reactor, we imported it earlier
reactor.run()

If the above cell code executed without any error and the daily bars length printed as last message with more than 0 elements then it means we were able to successfully get the daily trend bars data.

Now we have the price data, let'd do some cool stuff with it, we will transform it to a pandas DataFrame and then we will use it to train a sklearn model based on it.